In [1]:
from pandas import *

# must specify that blank space " " is NaN
experimentDF = pandas.read_csv('parasite_data.csv', na_values=[' '])

print(experimentDF.head())

   Virulence  Replicate  ShannonDiversity
0        0.5          1          0.059262
1        0.5          2          1.093600
2        0.5          3          1.139390
3        0.5          4          0.547651
4        0.5          5          0.065928


In [2]:
print(experimentDF.columns)

Index(['Virulence', 'Replicate', 'ShannonDiversity'], dtype='object')


In [3]:
print(experimentDF.ftypes)

Virulence           float64:dense
Replicate             int64:dense
ShannonDiversity    float64:dense
dtype: object


In [4]:
print(experimentDF['Virulence'].head())

0    0.5
1    0.5
2    0.5
3    0.5
4    0.5
Name: Virulence, dtype: float64


In [5]:
# 12th row
print(experimentDF["ShannonDiversity"][12])

1.58981


In [6]:
# Show all > 2.0
print(experimentDF[experimentDF['ShannonDiversity'] > 2.0].head())

     Virulence  Replicate  ShannonDiversity
8          0.5          9           2.04768
89         0.6         40           2.01066
92         0.6         43           2.90081
96         0.6         47           2.02915
105        0.7          6           2.23427


In [7]:
print(experimentDF[np.isnan(experimentDF["Virulence"])])

     Virulence  Replicate  ShannonDiversity
300        NaN          1          0.000000
301        NaN          2          0.000000
302        NaN          3          0.833645
303        NaN          4          0.000000
304        NaN          5          0.990309
305        NaN          6          0.000000
306        NaN          7          0.000000
307        NaN          8          0.000000
308        NaN          9          0.061414
309        NaN         10          0.316439
310        NaN         11          0.904773
311        NaN         12          0.884122
312        NaN         13          0.000000
313        NaN         14          0.000000
314        NaN         15          0.000000
315        NaN         16          0.000000
316        NaN         17          0.013495
317        NaN         18          0.882519
318        NaN         19          0.000000
319        NaN         20          0.986830
320        NaN         21          0.000000
321        NaN         22       

#### DataFrame methods automatically ignore NA/NaN values.

In [8]:
print("Mean virulence across all treatments:", experimentDF["Virulence"].mean())

Mean virulence across all treatments: 0.7500000000000013


#### However, not all methods in Python are guaranteed to handle NA/NaN values properly.

In [9]:
from scipy import stats  
  
print("Mean virulence across all treatments:", stats.sem(experimentDF["Virulence"])) 

Mean virulence across all treatments: nan


#### So we drop rows where any entry is NA/NaN

In [10]:
# NOTE: this drops the entire row if any of its entries are NA/NaN!  
print(experimentDF.dropna().count())

Virulence           300
Replicate           300
ShannonDiversity    300
dtype: int64


Alternatively, we could drop rows based on the content of a singe column

<code>print(experimentDF["Virulence"].dropna())</code>

Alternatively, we could fill NA/NaN locations with a value..

<code>print(experimentDF.fillna(0.0)["Virulence"]) </code>

In [11]:
print("Mean virulence across all treatments w/ dropped NaN:",  
    experimentDF["Virulence"].dropna().mean())  
  
print("Mean virulence across all treatments w/ filled NaN:",  
    experimentDF.fillna(0.0)["Virulence"].mean())

Mean virulence across all treatments w/ dropped NaN: 0.7500000000000013
Mean virulence across all treatments w/ filled NaN: 0.642857142857144


#### Mean of a dataset

In [12]:
print("Mean Shannon Diversity w/ 0.8 Parasite Virulence =", 
       experimentDF[experimentDF['Virulence'] == 0.8]['ShannonDiversity'].mean())

Mean Shannon Diversity w/ 0.8 Parasite Virulence = 1.2691338187999996


#### Variance in a data set

In [13]:
print("Mean Shannon Diversity w/ 0.8 Parasite Virulence =", 
       experimentDF[experimentDF['Virulence'] == 0.8]['ShannonDiversity'].var())

Mean Shannon Diversity w/ 0.8 Parasite Virulence = 0.6110384333126732


The standard deviation, or SD, measures the amount of variability or dispersion for a subject set of data from the mean, while the standard error of the mean, or <em>SEM</em>, measures how far the sample mean of the data is likely to be from the true population mean. The SEM is always smaller than the SD.

In [14]:
print("SEM of Shannon Diversity w/ 0.8 Parasite Virulence =",  
    stats.sem(experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]))

SEM of Shannon Diversity w/ 0.8 Parasite Virulence = 0.11054758552882764


#### Mann-Whitney-Wilcoxon (MWW) RankSum test
The MWW RankSum test is a useful test to determine if two distributions are significantly different or not. Unlike the t-test, the RankSum test does not assume that the data are normally distributed, potentially providing a more accurate assessment of the data sets.
As an example, let’s say we want to determine if the results of the two following treatments significantly differ or not:

In [15]:
# select two treatment data sets from the parasite data  
treatment1 = experimentDF[experimentDF["Virulence"] == 0.5]["ShannonDiversity"]  
treatment2 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]  
  
print("Data set 1:\n", treatment1.head())
print("Data set 2:\n", treatment2.head())

Data set 1:
 0    0.059262
1    1.093600
2    1.139390
3    0.547651
4    0.065928
Name: ShannonDiversity, dtype: float64
Data set 2:
 150    1.433800
151    2.079700
152    0.892139
153    2.384740
154    0.006980
Name: ShannonDiversity, dtype: float64


A RankSum test will provide a P value indicating whether or not the two distributions are the same.

In [16]:
z_stat, p_val = stats.ranksums(treatment1, treatment2)  
  
print("MWW RankSum P for treatments 1 and 2 =", p_val)  

MWW RankSum P for treatments 1 and 2 = 0.0009833559027350577


If P <= 0.05, we are highly confident that the distributions significantly differ, and can claim that the treatments had a significant impact on the measured value. If the treatments do not significantly differ, we could expect a result such as the following:

In [17]:
treatment3 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]  
treatment4 = experimentDF[experimentDF["Virulence"] == 0.9]["ShannonDiversity"]  
  
print("Data set 3:\n", treatment3.head())
print("Data set 4:\n", treatment4.head())

Data set 3:
 150    1.433800
151    2.079700
152    0.892139
153    2.384740
154    0.006980
Name: ShannonDiversity, dtype: float64
Data set 4:
 200    1.036930
201    0.938018
202    0.995956
203    1.006970
204    0.968258
Name: ShannonDiversity, dtype: float64


In [18]:
# compute RankSum P value  
z_stat, p_val = stats.ranksums(treatment3, treatment4)  
  
print("MWW RankSum P for treatments 3 and 4 =", p_val)

MWW RankSum P for treatments 3 and 4 = 0.9944995711242048


With P > 0.05, we must say that the distributions do not significantly differ. Thus changing the parasite virulence between 0.8 and 0.9 does not result in a significant change in Shannon Diversity.

#### One-way analysis of variance (ANOVA)
If you need to compare more than two data sets at a time, an ANOVA is your best bet. For example, we have the results from three experiments with overlapping 95% confidence intervals, and we want to confirm that the results for all three experiments are not significantly different.


In [19]:
treatment1 = experimentDF[experimentDF["Virulence"] == 0.7]["ShannonDiversity"]  
treatment2 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]  
treatment3 = experimentDF[experimentDF["Virulence"] == 0.9]["ShannonDiversity"]  
  
print("Data set 1:\n", treatment1.head())
print("Data set 2:\n", treatment2.head())
print("Data set 3:\n", treatment3.head())

Data set 1:
 100    1.595440
101    1.419730
102    0.000000
103    0.000000
104    0.787591
Name: ShannonDiversity, dtype: float64
Data set 2:
 150    1.433800
151    2.079700
152    0.892139
153    2.384740
154    0.006980
Name: ShannonDiversity, dtype: float64
Data set 3:
 200    1.036930
201    0.938018
202    0.995956
203    1.006970
204    0.968258
Name: ShannonDiversity, dtype: float64


In [20]:
# compute one-way ANOVA P value   
from scipy import stats  
      
f_val, p_val = stats.f_oneway(treatment1, treatment2, treatment3)  
  
print("One-way ANOVA P =", p_val) 

One-way ANOVA P = 0.3815094818741026


If P > 0.05, we can claim with high confidence that the means of the results of all three experiments are not significantly different.

#### Bootstrapped 95% confidence intervals

Oftentimes in wet lab research, it’s difficult to perform the 20 replicate runs recommended for computing reliable confidence intervals with SEM.

In this case, bootstrapping the confidence intervals is a much more accurate method of determining the 95% confidence interval around your experiment’s mean performance.

Unfortunately, SciPy doesn’t have bootstrapping built into its standard library yet. However, there is already a scikit out there for bootstrapping. Enter the following command to install it:

<code>sudo easy_install scikits.bootstrap</code>  

Bootstrapping 95% confidence intervals around the mean with this function is simple:

In [21]:
# subset a list of 10 data points  
treatment1 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"][:10]  
  
print("Small data set:\n", treatment1)

Small data set:
 150    1.433800
151    2.079700
152    0.892139
153    2.384740
154    0.006980
155    1.971760
156    0.000000
157    1.428470
158    1.715950
159    0.000000
Name: ShannonDiversity, dtype: float64


In [22]:
import scipy  
import scikits.bootstrap as bootstrap  
  
# compute 95% confidence intervals around the mean  
CIs = bootstrap.ci(data=treatment1, statfunction=scipy.mean)  
  
print("Bootstrapped 95% confidence intervals\nLow:", CIs[0], "\nHigh:", CIs[1])

Bootstrapped 95% confidence intervals
Low: 0.6137199720000001 
High: 1.7012658000000003


Note that you can change the range of the confidence interval by setting the alpha:

In [23]:
# 80% confidence interval  
CIs = bootstrap.ci(treatment1, scipy.mean, alpha=0.2)  
print("Bootstrapped 80% confidence interval\nLow:", CIs[0], "\nHigh:", CIs[1])

Bootstrapped 80% confidence interval
Low: 0.8195958999999998 
High: 1.5291377999999998


And also modify the size of the bootstrapped sample pool that the confidence intervals are taken from:

In [24]:
# bootstrap 20,000 samples instead of only 10,000  
CIs = bootstrap.ci(treatment1, scipy.mean, n_samples=20000)  
print("Bootstrapped 95% confidence interval w/ 20,000 samples\nLow:",  
    CIs[0], "\nHigh:", CIs[1]) 

Bootstrapped 95% confidence interval w/ 20,000 samples
Low: 0.614775824 
High: 1.692654024


Generally, bootstrapped 95% confidence intervals provide more accurate confidence intervals than 95% confidence intervals estimated from the SEM.